## SQL Server Connection Using SQLAlchemy

This project is designed to manage a SQL Server database using Python within a Jupyter Notebook environment. Here’s a comprehensive overview of what the project does:

1. **Establishing a Database Connection**:
   - The project begins by defining a function, `create_connection`, which establishes a connection to a SQL Server database using SQLAlchemy. This function takes parameters such as the server name, database name, and optionally, a username and password for SQL authentication. It constructs a connection string and attempts to connect to the database, returning an SQLAlchemy engine object if successful.

2. **Fetching and Displaying Data**:
   - Another function, `fetch_and_display_data`, is defined to execute SQL queries and fetch data from the connected database. This function uses the SQLAlchemy engine to run the provided SQL query and loads the results into a pandas DataFrame. The first 10 rows of the DataFrame are then displayed in a tabulated format using the `tabulate` library for better readability.

3. **Executing SQL Queries**:
   - The project includes examples of how to call these functions. For instance, it demonstrates how to connect to a specific SQL Server instance and database, and then how to fetch and display data from the `INFORMATION_SCHEMA.TABLES` and `sales` tables.

4. **Error Handling**:
   - Both functions include error handling to manage and report any issues that arise during the connection or data fetching processes. This ensures that users are informed of any problems and can take corrective actions.

Overall, this project provides a practical approach to managing and interacting with a SQL Server database using Python. It leverages the power of pandas for data manipulation and SQLAlchemy for database connectivity, making it a robust solution for database management tasks within a Jupyter Notebook.

### **Calling the SQL Server Connection Function**

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_connection(server, database, username=None, password=None):
    """
    Create a connection to the SQL Server database using SQLAlchemy.
    
    Parameters:
    - server: str, the name of the SQL Server
    - database: str, the name of the database
    - username: str, optional, SQL Server username (for SQL authentication)
    - password: str, optional, SQL Server password (for SQL authentication)
    
    Returns:
    - engine: sqlalchemy.engine.base.Engine, the SQLAlchemy engine object or None if the connection fails
    """
    try:
        if username and password:
            connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
        else:
            connection_string = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
        
        engine = create_engine(connection_string)
        
        # Test connection
        with engine.connect() as conn:
            logging.info("Connection successful!")
        
        return engine

    except SQLAlchemyError as e:
        logging.error(f"Connection failed: {e}")
        return None

In [3]:
def check_connection(engine):
    """
    Check if the database connection is successful.
    
    Parameters:
    - engine: sqlalchemy.engine.base.Engine, the SQLAlchemy engine object
    
    Returns:
    - bool: True if connection is successful, False otherwise
    """
    if engine:
        logging.info("Database connection established successfully.")
        return True
    else:
        logging.error("Failed to establish database connection.")
        return False

# Call the create_connection function
server = r'imsalione-pc\imsalionedb'
database = 'pubs'

engine = create_connection(server, database)

# Use the check_connection function to verify the connection
if check_connection(engine):
    # Proceed with database operations
    pass
else:
    # Handle the failed connection
    pass

2024-09-22 00:21:54,997 - INFO - Connection successful!
2024-09-22 00:21:54,999 - INFO - Database connection established successfully.


### **Fetching and Displaying Data from SQL Server**

In [4]:
import pandas as pd
from sqlalchemy import exc
from pathlib import Path
from openpyxl import load_workbook

# Define the directory where you want to save the Excel file
output_directory = Path("C:/Reports")  # Adjust the path as needed
output_directory.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

def fetch_and_display_data(engine, query, pnum=10, export_to_excel=False):
    """
    Fetch data from a SQL database and optionally export to Excel.
    
    Parameters:
    - engine: SQLAlchemy engine object.
    - query: SQL query string.
    - pnum: int, number of rows to display (default is 10).
    - export_to_excel: bool, whether to export data to an Excel file (default is False).
    - excel_output_name: str, name of the Excel file to export (default is 'output.xlsx').
    """
    
    if engine:
        try:
            # Load data into a pandas DataFrame
            df = pd.read_sql(query, engine)
            
            # Display the first pnum rows (or default 10 rows if pnum is not provided)
            print(df.head(pnum).to_string(index=False))

            # Export to Excel if requested
            if export_to_excel:
                # Combine directory path with file name
                file_path = output_directory / "output.xlsx"

                # Check if the Excel file already exists
                if file_path.exists():
                    try:
                        # Append new sheet to the existing workbook
                        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
                            new_sheet_name = f"Sheet_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
                            df.to_excel(writer, sheet_name=new_sheet_name, index=False)
                            print(f"Data exported to {file_path} in sheet {new_sheet_name}")
                    except Exception as e:
                        print(f"Error writing to existing Excel file: {e}")
                else:
                    # Create a new Excel file and add the data to the first sheet
                    df.to_excel(file_path, sheet_name='Sheet1', index=False)
                    print(f"Data exported to {file_path} in sheet 'Sheet1'")

        except exc.SQLAlchemyError as db_error:
            print(f"Database error occurred: {db_error}")
        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print("No database connection.")

### Executing SQL Query to Fetch and Display Table Information

In [6]:
query = "SELECT * FROM INFORMATION_SCHEMA.TABLES"
fetch_and_display_data(engine, query, pnum=10, export_to_excel=True)

TABLE_CATALOG TABLE_SCHEMA  TABLE_NAME TABLE_TYPE
         pubs          dbo sysdiagrams BASE TABLE
         pubs          dbo     authors BASE TABLE
         pubs          dbo  publishers BASE TABLE
         pubs          dbo      titles BASE TABLE
         pubs          dbo titleauthor BASE TABLE
         pubs          dbo      stores BASE TABLE
         pubs          dbo    roysched BASE TABLE
         pubs          dbo   discounts BASE TABLE
         pubs          dbo        jobs BASE TABLE
         pubs          dbo    pub_info BASE TABLE
Data exported to C:\Reports\output.xlsx in sheet 'Sheet1'


### Executing SQL Query to Fetch and Display Sales Data

In [10]:
query = "SELECT * FROM sales"
fetch_and_display_data(engine, query, 20, False)

 id stor_id  ord_num   ord_date  qty   payterms title_id
  1    6380     6871 1994-09-14    5     Net 60   BU1032
  2    6380     722a 1994-09-13    3     Net 60   PS2091
  3    7066    A2976 1993-05-24   50     Net 30   PC8888
  4    7066 QA7442.3 1994-09-13   75 ON invoice   PS2091
  5    7067    D4482 1994-09-14   10     Net 60   PS2091
  6    7067    P2121 1992-06-15   40     Net 30   TC3218
  7    7067    P2121 1992-06-15   20     Net 30   TC4203
  8    7067    P2121 1992-06-15   20     Net 30   TC7777
  9    7131  N914008 1994-09-14   20     Net 30   PS2091
 10    7131  N914014 1994-09-14   25     Net 30   MC3021
 11    7131   P3087a 1993-05-29   20     Net 60   PS1372
 12    7131   P3087a 1993-05-29   25     Net 60   PS2106
 13    7131   P3087a 1993-05-29   15     Net 60   PS3333
 14    7131   P3087a 1993-05-29   25     Net 60   PS7777
 15    7896   QQ2299 1993-10-28   15     Net 60   BU7832
 16    7896    TQ456 1993-12-12   10     Net 60   MC2222
 17    7896     X999 1993-02-21

In [11]:
query = """SELECT *, CASE
WHEN qty < 30 THEN N'کم فروش'
WHEN qty >= 30 THEN N'پرفروش'
END AS SalesClass 
FROM sales"""

fetch_and_display_data(engine, query, 10, False)

 id stor_id  ord_num   ord_date  qty   payterms title_id SalesClass
  1    6380     6871 1994-09-14    5     Net 60   BU1032    کم فروش
  2    6380     722a 1994-09-13    3     Net 60   PS2091    کم فروش
  3    7066    A2976 1993-05-24   50     Net 30   PC8888     پرفروش
  4    7066 QA7442.3 1994-09-13   75 ON invoice   PS2091     پرفروش
  5    7067    D4482 1994-09-14   10     Net 60   PS2091    کم فروش
  6    7067    P2121 1992-06-15   40     Net 30   TC3218     پرفروش
  7    7067    P2121 1992-06-15   20     Net 30   TC4203    کم فروش
  8    7067    P2121 1992-06-15   20     Net 30   TC7777    کم فروش
  9    7131  N914008 1994-09-14   20     Net 30   PS2091    کم فروش
 10    7131  N914014 1994-09-14   25     Net 30   MC3021    کم فروش


In [12]:
query = """SELECT * , DATEDIFF(YEAR, hire_date, GETDATE())
                       AS experience 
                       FROM employee"""

fetch_and_display_data(engine, query , 10, False)

   emp_id     fname minit     lname  job_id  job_lvl pub_id  hire_date  experience
PMA42628M     Paolo     M   Accorti      13       35   0877 1992-08-27          32
PSA89086M     Pedro     S    Afonso      14       89   1389 1990-12-24          34
VPA30890F  Victoria     P  Ashworth       6      140   0877 1990-09-13          34
H-B39728F     Helen         Bennett      12       35   0877 1989-09-21          35
L-B31947F    Lesley           Brown       7      120   0877 1991-02-13          33
F-C16315M Francisco           Chang       4      227   9952 1990-11-03          34
PTC11962M    Philip     T    Cramer       2      215   9952 1989-11-11          35
A-C71970F      Aria            Cruz      10       87   1389 1991-10-26          33
AMD15433F       Ann     M     Devon       3      200   9952 1991-07-16          33
ARD36773F   Anabela     R Domingues       8      100   0877 1993-01-27          31


In [13]:
query = """SELECT fname, lname, hire_date , DATEDIFF(YEAR, hire_date, GETDATE())
                       AS experience, 
                       CASE 
                       WHEN DATEDIFF(YEAR, hire_date, GETDATE()) > 30 THEN 'G1'
                       WHEN DATEDIFF(YEAR, hire_date, GETDATE()) <= 30 THEN 'G2'
                       END AS experienceClass
                       FROM employee"""

fetch_and_display_data(engine, query , 10, False)

    fname     lname  hire_date  experience experienceClass
    Paolo   Accorti 1992-08-27          32              G1
    Pedro    Afonso 1990-12-24          34              G1
 Victoria  Ashworth 1990-09-13          34              G1
    Helen   Bennett 1989-09-21          35              G1
   Lesley     Brown 1991-02-13          33              G1
Francisco     Chang 1990-11-03          34              G1
   Philip    Cramer 1989-11-11          35              G1
     Aria      Cruz 1991-10-26          33              G1
      Ann     Devon 1991-07-16          33              G1
  Anabela Domingues 1993-01-27          31              G1


In [14]:
query = """SELECT * FROM sales
WHERE title_id like 'B%'"""

fetch_and_display_data(engine, query , 20, False)

 id stor_id  ord_num   ord_date  qty   payterms title_id
  1    6380     6871 1994-09-14    5     Net 60   BU1032
 15    7896   QQ2299 1993-10-28   15     Net 60   BU7832
 17    7896     X999 1993-02-21   35 ON invoice   BU2075
 19    8042 423LL930 1994-09-14   10 ON invoice   BU1032
 20    8042     P723 1993-03-11   25     Net 30   BU1111


In [33]:
query = """SELECT title, type, isnull(price, 0)
AS NewPrice
FROM TITLES
ORDER BY NewPrice"""

fetch_and_display_data(engine, query , 10, False)

                                               title         type  NewPrice
                  The Psychology of Computer Cooking UNDECIDED         0.00
                                       Net Etiquette popular_comp      0.00
                               The Gourmet Microwave mod_cook          2.99
                     You Can Combat Computer Stress! business          2.99
                                   Life Without Fear psychology        7.00
                 Emotional Security: A New Algorithm psychology        7.99
                                 Is Anger the Enemy? psychology       10.95
           Fifty Years in Buckingham Palace Kitchens trad_cook        11.95
Cooking with Computers: Surreptitious Balance Sheets business         11.95
                                      Sushi, Anyone? trad_cook        14.99
